In [27]:
from bs4 import BeautifulSoup
from requests import get
import random
import string
import sqlite3
import logging

# Input
flickfilosopher_file = '../Scraping/flickfilosopher.txt'
current_reviews = set(line.strip() for line in open(flickfilosopher_file))

# Output
# log_path = '../Logs/movieReviews.log'
db_path = '../Data/test.db'
db_name = 'test'
conn = sqlite3.connect(db_path)
c = conn.cursor()

# logger = logging.getLogger()
# fhandler = logging.FileHandler(filename=log_path, mode='a')
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# fhandler.setFormatter(formatter)
# logger.addHandler(fhandler)
# logger.setLevel(logging.DEBUG)

def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

review_count = 0
no_annotation = 0
no_rating = 0

review_link ="https://www.flickfilosopher.com/2016/02/the-act-of-killing-and-the-look-of-silence-documentaries-review-when-horror-doesnt-destroy-humanity.html"
if review_link is not None:
    if review_link.endswith('html'):
        if review_link in current_reviews:
            pass
        else:
            review_soup = BeautifulSoup(get(review_link).text, 'lxml')
            review_html = review_soup.find("div", itemtype="http://schema.org/Review")
            reviewBody = ""
            if review_html is None:
                no_annotation += 1
            else:
                try:
                    reviewBody_html = review_html.find_all("div", class_="reviewtext")
                    reviewRating = review_soup.find("div", itemtype="http://schema.org/Rating")
                    for item in reviewBody_html:
                        reviewBody += item.p.text
                    worstRating_tag = reviewRating.find("meta", itemprop="worstRating")
                    bestRating_tag = reviewRating.find("meta", itemprop="bestRating")
                    ratingValue_tag = reviewRating.find("meta", itemprop="ratingValue")
                    if worstRating_tag is None:
                        worstRating = reviewRating.find("div", itemprop="worstRating")["content"]
                        bestRating = reviewRating.find("div", itemprop="bestRating")["content"]
                        ratingValue = reviewRating.find("div", itemprop="ratingValue")["content"]
                    else:
                        worstRating = worstRating_tag["content"]
                        bestRating = bestRating_tag["content"]
                        ratingValue = ratingValue_tag["content"]
                    if ratingValue == "":
                        no_rating += 1
                    else:
                        node = generateNode(31)
                        c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
                        conn.commit()
                        review_count += 1
                except Exception as e:
                    raise(e)
                    print("Error: " + str(review_link))


print("amount of reviews extracted: " + str(review_count) + ", amount of entries without review annotations: " + str(no_annotation) + " amount of entries without rating: " + str(no_rating))





<div itemprop="reviewRating" itemscope="" itemtype="http://schema.org/Rating">
<div content="5" itemprop="bestRating">
<div content="0" itemprop="worstRating">
<div content="5" itemprop="ratingValue">
</div>
</div>
</div>
</div>
amount of reviews extracted: 1, amount of entries without review annotations: 0 amount of entries without rating: 0
